In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [4]:
path_to_file

'/root/.keras/datasets/shakespeare.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

len(text)

1115394

In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
vocab = sorted(set(text))

In [8]:
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [10]:
# 텍스트에서 처음 13개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [11]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [12]:
seq_length = 100
examples_per_epoch = len(text) // seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
        whether the last batch should be dropped in the case it has fewer than
        `batch_size` elements; the default behavior is not to drop the smaller
    batch.

In [13]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [16]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [17]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(len(vocab), embedding_dim, rnn_units, BATCH_SIZE)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [38]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 65), dtype=float32, numpy=
array([[-0.00345868,  0.00580432,  0.00428756, ..., -0.00469717,
         0.00302682,  0.00848159],
       [ 0.00214205,  0.00093147, -0.0039704 , ..., -0.00661413,
        -0.00372979,  0.00720337],
       [-0.00344468,  0.00111368, -0.0040472 , ..., -0.00355963,
         0.00033616,  0.00846841],
       ...,
       [ 0.00186549,  0.00215303,  0.00199312, ..., -0.0110195 ,
         0.00083113,  0.00725814],
       [-0.00056774,  0.00623655,  0.00254553, ..., -0.01521521,
        -0.00267771,  0.00756258],
       [-0.0032464 ,  0.00418563,  0.00081967, ..., -0.00739336,
         0.00260251,  0.00831849]], dtype=float32)>

In [39]:
sampled_indices

array([ 6, 37, 61, 61, 61, 30, 59, 31,  0, 34, 44,  2,  2, 58, 23,  8, 14,
        9, 47, 39, 31, 41, 55, 13, 11, 57, 26,  8, 15, 48,  0, 11,  8, 33,
       34, 45, 20, 45, 59, 32, 33, 32, 39, 52, 58, 36, 41,  6, 41,  4, 20,
       60, 11, 26, 14, 14, 60,  8, 37, 43, 17, 12, 11, 57, 16, 26, 24,  3,
       37, 62, 23, 49, 30, 11, 45, 12, 22, 20, 32,  1,  3, 40, 19, 25, 55,
       44, 18, 51, 10,  1, 53, 64, 59, 30, 50,  9, 26, 37, 54, 17])

In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(sampled_indices.shape)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
print(sampled_indices.shape)

(100, 1)
(100,)


In [25]:
sampled_indices

array([ 6, 37, 61, 61, 61, 30, 59, 31,  0, 34, 44,  2,  2, 58, 23,  8, 14,
        9, 47, 39, 31, 41, 55, 13, 11, 57, 26,  8, 15, 48,  0, 11,  8, 33,
       34, 45, 20, 45, 59, 32, 33, 32, 39, 52, 58, 36, 41,  6, 41,  4, 20,
       60, 11, 26, 14, 14, 60,  8, 37, 43, 17, 12, 11, 57, 16, 26, 24,  3,
       37, 62, 23, 49, 30, 11, 45, 12, 22, 20, 32,  1,  3, 40, 19, 25, 55,
       44, 18, 51, 10,  1, 53, 64, 59, 30, 50,  9, 26, 37, 54, 17])

In [27]:
print(repr("".join(idx2char[sampled_indices])))

',YwwwRuS\nVf!!tK.B3iaScqA;sN.Cj\n;.UVgHguTUTantXc,c&Hv;NBBv.YeE?;sDNL$YxKkR;g?JHT $bGMqfFm: ozuRl3NYpE'


In [28]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print(example_batch_predictions.shape)
print(example_batch_loss.numpy().mean())

(64, 100, 65)
4.173069


In [30]:
example_batch_loss.shape

TensorShape([64, 100])

In [31]:
model.compile(optimizer='adam', loss=loss)

In [32]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 7s 42ms/step - loss: 2.6094
Epoch 2/10
172/172 [==============================] - 7s 42ms/step - loss: 1.9071
Epoch 3/10
172/172 [==============================] - 7s 42ms/step - loss: 1.6559
Epoch 4/10
172/172 [==============================] - 7s 42ms/step - loss: 1.5184
Epoch 5/10
172/172 [==============================] - 7s 42ms/step - loss: 1.4343
Epoch 6/10
172/172 [==============================] - 7s 42ms/step - loss: 1.3771
Epoch 7/10
172/172 [==============================] - 7s 42ms/step - loss: 1.3312
Epoch 8/10
172/172 [==============================] - 7s 42ms/step - loss: 1.2929
Epoch 9/10
172/172 [==============================] - 7s 42ms/step - loss: 1.2575
Epoch 10/10
172/172 [==============================] - 7s 42ms/step - loss: 1.2237


In [34]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [36]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [40]:
def generate_text(model, start_string):
    # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

    # 생성할 문자의 수
    num_generate = 1000

    # 시작 문자열을 숫자로 변환(벡터화)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # 결과를 저장할 빈 문자열
    text_generated = []

    # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
    # 온도가 높으면 더 의외의 텍스트가 됩니다.
    # 최적의 세팅을 찾기 위한 실험
    temperature = 1.0

    # 여기에서 배치 크기 == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # 배치 차원 제거
        predictions = tf.squeeze(predictions, 0)

        # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 예측된 단어를 다음 입력으로 모델에 전달
        # 이전 은닉 상태와 함께
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [41]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: they say,
or both I may.

PROSPERO:
He need made gone I not sat you now? I have
A pardon nich an oppos possessed
Against thou harst from yourselves;
To fault how he was the general
O' the LeEORENCE:
I do learned a holy of a low;
And will you not mean but with my father's swifl
Sign of his chark o' the ladies, gaunts not
with you.

QUEEN ELIZABETH:
Upon as Volscians: I will corn a
sayet: if it presently be rule
Of noble Gauch of Dian's whether:
I cannot do, and if I let your block-way,
anch I behold an althoke evil wings re up, sleep
To be Tybll knee.

FRETRUCHIO:
Tut, I saw short; an his way will grant him.

VILARDAANE:
I must pircuse him to revenge him of the face,
Nor when thou hast act to wime a man allow the crown?
I lean to unsay you are too fair
Trike wars.

Servant:
I am advised, but what's the exquest duked.

Second Citizen:
Well most I will, good nothing.
Marry, then, look one that is some regal to them,
Unkindly do no man to scorn Aares.

PETRUCHIO:
Tublish, Padua, Pom

In [42]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

optimizer = tf.keras.optimizers.Adam()

In [43]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    return loss

In [44]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    hidden = model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)
        
        if batch_n % 100 == 0:
            template = 'epoch {} batch {} loss {}'
            print(template.format(epoch + 1, batch_n, loss))
            
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
        
    print ('에포크 {} 손실 {:.4f}'.format(epoch+1, loss))
    print ('1 에포크 당 {}초 소요\n'.format(time.time() - start))
    
model.save_weights(checkpoint_prefix.format(epoch=epoch))

epoch 1 batch 0 loss 4.17349100112915
epoch 1 batch 100 loss 2.330228805541992
에포크 1 손실 2.0812
1 에포크 당 8.316502094268799초 소요

epoch 2 batch 0 loss 2.106487512588501
epoch 2 batch 100 loss 1.8289170265197754
에포크 2 손실 1.7290
1 에포크 당 7.497234582901001초 소요

epoch 3 batch 0 loss 1.77058744430542
epoch 3 batch 100 loss 1.5786494016647339
에포크 3 손실 1.5419
1 에포크 당 7.441206693649292초 소요

epoch 4 batch 0 loss 1.5963642597198486
epoch 4 batch 100 loss 1.5338521003723145
에포크 4 손실 1.4496
1 에포크 당 7.64025354385376초 소요

epoch 5 batch 0 loss 1.4337985515594482
epoch 5 batch 100 loss 1.4329538345336914
에포크 5 손실 1.4005
1 에포크 당 7.6407506465911865초 소요

epoch 6 batch 0 loss 1.3625717163085938
epoch 6 batch 100 loss 1.3529794216156006
에포크 6 손실 1.3437
1 에포크 당 7.525149345397949초 소요

epoch 7 batch 0 loss 1.300917387008667
epoch 7 batch 100 loss 1.3230106830596924
에포크 7 손실 1.3162
1 에포크 당 7.576035976409912초 소요

epoch 8 batch 0 loss 1.27394700050354
epoch 8 batch 100 loss 1.2997967004776
에포크 8 손실 1.2850
1 에포크 당 7.5

In [47]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I'll bury, I think it switnell Thut can this:
There are no store doubling forthtainsly rich in
the people impress of Ricilene'full bourney
Were he look'd to myself! so panity, nor now?
Take him the cofe; I'll seem them iz, and never grie.

BUCKINGHAM:
It is not have been sucutited now:
Speak on, let us shame of liberty, wive believed,
Accountensed men.

LUCIO:
Well, gentle sirs,
Where's a helfour mark, like England's crown.
Did doom upon more beauty's love, but for this good
King to approach into thy danger?
O slave witnes to't, o'erless your cousperser;
But now I will been time may make thee go.

RUTLANIO:
I never
The engreaties like a tife.

ISABELLA:
If thou chear? it is! Look you shall possess.
Some one is an your eyes o' the be:
Move mine armour of my shame, dost book on your birth;
A lord of Clarence, now thy wills delivered, untent
it: and now I come, I speak but home,
Accubment hold of yonour noble wails,
With colouries of brief-tiged
Till thou wilt come to me: Clarence?

In [48]:
EPOCHS = 30

for epoch in range(EPOCHS):
    start = time.time()
    hidden = model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)
        
        if batch_n % 100 == 0:
            template = 'epoch {} batch {} loss {}'
            print(template.format(epoch + 1, batch_n, loss))
            
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
        
    print ('에포크 {} 손실 {:.4f}'.format(epoch+1, loss))
    print ('1 에포크 당 {}초 소요\n'.format(time.time() - start))
    
model.save_weights(checkpoint_prefix.format(epoch=epoch))

epoch 1 batch 0 loss 1.1109122037887573
epoch 1 batch 100 loss 1.2117351293563843
에포크 1 손실 1.2207
1 에포크 당 7.649111986160278초 소요

epoch 2 batch 0 loss 1.1149808168411255
epoch 2 batch 100 loss 1.1696817874908447
에포크 2 손실 1.2061
1 에포크 당 7.633092641830444초 소요

epoch 3 batch 0 loss 1.0897119045257568
epoch 3 batch 100 loss 1.1215019226074219
에포크 3 손실 1.1509
1 에포크 당 7.685880184173584초 소요

epoch 4 batch 0 loss 1.0361850261688232
epoch 4 batch 100 loss 1.1062809228897095
에포크 4 손실 1.0733
1 에포크 당 7.5650529861450195초 소요

epoch 5 batch 0 loss 1.0132596492767334
epoch 5 batch 100 loss 1.0301814079284668
에포크 5 손실 1.0256
1 에포크 당 7.60781192779541초 소요

epoch 6 batch 0 loss 0.9877865314483643
epoch 6 batch 100 loss 1.0196856260299683
에포크 6 손실 1.0227
1 에포크 당 7.67925238609314초 소요

epoch 7 batch 0 loss 0.927582859992981
epoch 7 batch 100 loss 0.9689609408378601
에포크 7 손실 1.0065
1 에포크 당 7.72675895690918초 소요

epoch 8 batch 0 loss 0.8837587237358093
epoch 8 batch 100 loss 0.9384923577308655
에포크 8 손실 0.9527
1 

In [50]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I
Think for I for wile untly statuee upon
Of death and rotten sister of my council: and then currents
Our roser: how they setting that you pluck
Your cold commanmed to him?

ANTONIO:
Well, Signior Gremio!

PARDINA:
Sir, a while, do so?

AUFIDIUS:
Mark'd you this instrument,
With pute must kies unhappier of vewnry.
The castles shempel blushing glass partic,
That, Pompey number wander of this love,
And that may reason him, I have been
As dring a citizease Inelful baggance.
Thy trareful lambs I cannot rule.

ROMEO:
My lord,
It boy'd to melt, the whoe are quainted that fith you!
Afflicts, here to hims my lord wives right:
Thou couldst upon her, for my death we heard.

LADY GREY:
Where are the which, and the earth with him, I say: therefore, though we't
Haste apperial bulks, puts to your hand, without sorrow,
Tor mounty man to die, and have with'd before: I will: what
Was my tongue fulling ost of such as a from mouth o'clock?

KING RICHARD IIA:
Your Musterer, I did not rouse again.



In [51]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: when I
gaily world? and you have more greater of venteance, day, nor I can
then in the cause I am to crow for no most;
And, nor no jest.

PETRUCHIO:
Good must be no borve hither by the devilder,
Creet Menenius.

Mostague, and first till case from hate
And met the tyren of Rome.

JOKN OF GAUNT:
But how you have we themen, whose content me down?

KING EDWARD IV:
Naw, madraws are gone; and would young witness for ever
coll and rule, was a loss and contenanced was't.

Third Gentleman:
Unread our great days?
I kids to queat Godfolk, Signior Gremio;
I shall not bad any certail of the
nambling footth so faster;' 'Me swelt so,
And you, my lord, thy life did scept to 't.
Now, mistress cousin and to sleep aidsh
In stofy one: O, all what to Warwick?

JULIET:
You shall, emprished me
But yether lord, I'll bring thee thine eyes.

JOHN OF GAUNT:
Thou art good Ratharn'd thought the Strives i' them;
And thou hollonger for what we thee ' made word 'banished,'
And such welcome, depry, and any whit